In [4]:
from numpy import load
import numpy as np
import pandas as pd
import itertools
import requests
from io import StringIO
import pyarrow.parquet as pq
from urllib.error import HTTPError
import warnings

In [2]:
acceptable_columns = ['address', 'age', 'area', 'birth Date', 'birth Place', 'brand', 'city', 'continent', 'country', 'county', 'currency', 'day', 'duration', 'industry', 'language', 'location', 'manufacturer', 'name', 'nationality', 'order', 'person', 'product', 'range', 'rank', 'region','sales', 'sex', 'state', 'status', 'symbol', 'type', 'year']
acceptable_columns = np.array(acceptable_columns)
# np.save("classes_sherlock.npy", acceptable_columns, allow_pickle=False)


In [3]:
# main dataframes to add onto existing dataframe
new_train = pd.DataFrame(columns = ['type', 'values'])
new_test = pd.DataFrame(columns = ['type', 'values'])
new_val = pd.DataFrame(columns = ['type', 'values'])

# list of urls of csvs
urls = ['https://drive.google.com/file/d/1VBEYn2bV9hUg6Vs97StMMR8NudP--v-D/view?usp=sharing', 'https://drive.google.com/file/d/1XoHPXvb_eAH-90jTSj-1sePVzQtIlw-n/view?usp=sharing',
       'https://drive.google.com/file/d/1QwjBOKHSYbEqZxDeUfVZVOjkgb_xuaUt/view?usp=sharing', 'https://drive.google.com/file/d/1I4btyUubuYR6sgz2YhfEftNXdJuKaPaN/view?usp=sharing',
       'https://drive.google.com/file/d/1X6B-9hRgENR4xAjL1d0m6BmxGZUs6eGF/view?usp=sharing', 'https://drive.google.com/file/d/1DiHNQZUb9icUhkeBvWYyARKYscDPUAyx/view?usp=sharing',
       'https://drive.google.com/file/d/1GhY41caqsjYhTVdUaHqyzp6b-bUJSiX9/view?usp=sharing', 'https://drive.google.com/file/d/1ISLN-fra-7rzfar4b8LvM20YF_UNWC52/view?usp=sharing',
       'https://drive.google.com/file/d/140MKfvrMnIy-sfNlovGmHShEETBagMBw/view?usp=sharing', 'https://drive.google.com/file/d/1JPdeXZ5h6zM_esbpUwlpVD-hFx7ifVmm/view?usp=sharing', 
       'https://drive.google.com/file/d/10eD7XzIql6LOVJlW9C7Wzc1DVnECigBw/view?usp=sharing', 'https://drive.google.com/file/d/1IfQ5SAPgdX8R84qvWxAIy-XoMGC4EuNW/view?usp=sharing',
       'https://drive.google.com/file/d/1bHYZGXgYP7PYWrNFvpXGJXKJNgZxhyBU/view?usp=sharing']


# list of types
types = ['naic', 'credit card account numbers', 'credit card account numbers', 'credit card account numbers', 'fips code', 'lei',
        'mcc code', 'phone number', 'zip code', 'guuid','guuid','tax_id','social_security']

In [5]:
#loop through all urls
for i in range(len(urls)):
    url = urls[i]
    #check that url is valid
    try:
        file_id = url.split('/')[-2]
        dwn_url='https://drive.google.com/uc?id=' + file_id
        #check that data is csv
        try:
            data = pd.read_csv(dwn_url, sep=";", header=None)
            t = types[i]
            #check that data is single column
            if data.shape[1] != 1:
                print('Data is not a single column')
                continue
        except UnicodeDecodeError:
            print('Data not in csv format')
            continue
    except HTTPError:
        print('Error: Link '+ url + ' not valid')
        continue
    
    # split 0.8 to training, 0.1 to test, and 0.1 to validation
    train_idx = round(len(data)*0.8)
    test_idx = round(len(data)*0.15)
    train_data = data.iloc[:train_idx]
    test_data = data.iloc[train_idx: (train_idx+test_idx)]
    val_data = data.iloc[(train_idx+test_idx):]

    # splitting into chunks of ~five data points
    train_data = train_data.values.tolist()
    #split data from list
    train_data = list(itertools.chain(*train_data))
    splitval = train_idx/5
    train_data = [str(i) for i in train_data]
    #split into 5 seperate arrays
    train_data_split = np.array_split(train_data, splitval)
    #gather arrays into dataframe
    train_df = pd.Series(train_data_split, name='values').to_frame()
    #set train_df['type'] to current type
    train_df['type'] = t
    #convert all train_df['values'] to string
    train_df = train_df.iloc[:,[1,0]]
    train_df['values'] = [str(i) for i in train_df['values']]

    #repeat above for test_data
    splitval = round(test_idx/5)
    test_data = test_data.values.tolist()
    test_data = list(itertools.chain(*test_data))
    test_data = [str(i) for i in test_data]
    test_data_split = np.array_split(test_data, splitval)
    test_df = pd.Series(test_data_split, name='values').to_frame()
    test_df['type'] = t
    test_df = test_df.iloc[:,[1,0]]
    test_df['values'] = [str(i) for i in test_df['values']]

    #repeat above for validation_data
    splitval = round(len(val_data)/5)
    val_data = val_data.values.tolist()
    val_data = list(itertools.chain(*val_data))
    val_data = [str(i) for i in val_data]
    val_data_split = np.array_split(val_data, splitval)
    val_df = pd.Series(val_data_split, name='values').to_frame()
    val_df['type'] = t
    val_df = val_df.iloc[:,[1,0]] 
    val_df['values'] = [str(i) for i in val_df['values']]
    
    #remove warnings for append dataframe
    warnings.filterwarnings('ignore')
    
    # add to main dataframes
    new_train = new_train.append(train_df, ignore_index=True)
    new_test = new_test.append(test_df, ignore_index=True)
    new_val = new_val.append(val_df, ignore_index=True)

In [10]:
# pull original sherock testing set
test_df_values = pq.read_table(source="./data/raw/test_values.parquet").to_pandas()
test_df_labels = pq.read_table(source="./data/raw/test_labels.parquet").to_pandas()
test_df_full = test_df_labels.merge(test_df_values, left_index=True, right_index=True)
test_df_full = test_df_full[test_df_full['type'].isin(acceptable_columns)]
# add new testing data
test_df_full = test_df_full.append(new_test, ignore_index=True)
test_df_values = pd.DataFrame(test_df_full, columns=['values'])
test_df_labels = pd.DataFrame(test_df_full, columns=['type'])
len(test_df_full.iloc[5])

2

In [8]:
# pull original sherock training set
train_df_values = pq.read_table(source="./data/raw/train_values.parquet").to_pandas()
train_df_labels = pq.read_table(source="./data/raw/train_labels.parquet").to_pandas()
train_df_full = train_df_labels.merge(train_df_values, left_index=True, right_index=True)
train_df_full = train_df_full[train_df_full['type'].isin(acceptable_columns)]
# add new training data
train_df_full = train_df_full.append(new_train, ignore_index=True)
train_df_values = pd.DataFrame(train_df_full, columns=['values'])
train_df_labels = pd.DataFrame(train_df_full, columns=['type'])
train_df_full.loc[train_df_full["type"] == "zip code"]

,type,values
201586,zip code,['36003' '36006' '36067' '36066' '36703' '36701']
201587,zip code,['36091' '36051' '36068' '36008' '36022']
201588,zip code,['36749' '36758' '36550' '36551' '36527']
201589,zip code,['36577' '36559' '36536' '36576' '36579']
201590,zip code,['36502' '36578' '36533' '36564' '36561']
...,...,...
210043,zip code,['57106' '57110' '57109' '57055' '57005']
210044,zip code,['57030' '57033' '57104' '57118' '57107']
210045,zip code,['57020' '57105' '57041' '57003' '57048']
210046,zip code,['57186' '57198' '57015' '57103' '57022']


In [12]:
# pull original sherock validation set
val_df_values = pq.read_table(source="./data/raw/val_values.parquet").to_pandas()
val_df_labels = pq.read_table(source="./data/raw/val_labels.parquet").to_pandas()
val_df_full = val_df_labels.merge(val_df_values, left_index=True, right_index=True)
val_df_full = val_df_full[val_df_full['type'].isin(acceptable_columns)]
# add new validation data
val_df_full = val_df_full.append(new_val, ignore_index=True)
val_df_values = pd.DataFrame(val_df_full, columns=['values'])
val_df_labels = pd.DataFrame(val_df_full, columns=['type'])
val_df_full

,type,values
0,county,"['Shelby', 'Knox', 'Washington', 'Davidson', '..."
1,age,"[20.42, 23.17, 24.5, 24.0, 21.92, 28.08, 25.5,..."
2,address,"['San Francisco, CA 94102', 'San Francisco, CA..."
3,duration,"['Last 7 Games', 'Last 15 Games', 'Last 30 Gam..."
4,industry,"['State government (OES designation)', 'Busine..."
...,...,...
66130,social_security,['213 18 5932' '100-80-6113' '913 29 5822' '80...
66131,social_security,['136-31-8040' '474-96-8926' '486 66 5129' '85...
66132,social_security,['187 69 1909' '518-01-9530' '355-56-0529' '84...
66133,social_security,['807-13-9017' '752-05-3696' '422-85-0475' '29...


In [15]:
#save test,train,and validation labels to data folder
test_df_labels.to_parquet("./last_data/raw/test_labels.parquet", engine='fastparquet')
test_df_values.to_parquet("./last_data/raw/test_values.parquet", engine='fastparquet')

train_df_labels.to_parquet("./last_data/raw/train_labels.parquet", engine='fastparquet')
train_df_values.to_parquet("./last_data/raw/train_values.parquet", engine='fastparquet')

val_df_labels.to_parquet("./last_data/raw/val_labels.parquet", engine='fastparquet')
val_df_values.to_parquet("./last_data/raw/val_values.parquet", engine='fastparquet')

